# Testing Architectures of the Neural Networks

Using the refactored code within the dnn/models/ directory, we want to go through each model and ensure that the models meet certain standards.

*Consider replacing with unit testing*

In [1]:
import sys
sys.path.append('../../dnn/')
from model.nets.ieegcnn import iEEGCNN
from model.nets.ieegseq import iEEGSeq
import model.train
from model.train import traincnn, trainseq

import processing.util as util

import time
import numpy as np
# np.random.seed(1234)
from functools import reduce
import math as m
import os

# import DNN frameworks
import tensorflow as tf
import keras

# import high level optimizers, models and layers
from keras.optimizers import SGD
from keras.preprocessing import sequence
from keras.layers.embeddings import Embedding

# utility for datasets and training
# from keras.utils.training_utils import multi_gpu_model
from keras.datasets import imdb
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
# imports tensorflow
# from keras import backend as K

from keras.utils import plot_model
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot


%load_ext autoreload
%autoreload 2

/Users/adam2392/Documents/dnn-unsupervised/.venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
imsize=32
n_colors =3 
num_classes=2
modeldim=2
DROPOUT=True

cnn = iEEGCNN(imsize=imsize,
              n_colors=n_colors, 
              num_classes=num_classes, 
              modeldim=modeldim, 
              DROPOUT=DROPOUT)
cnn.buildmodel()
cnn.summaryinfo()
print(cnn.model.input_shape)
print(cnn.model.summary())

{'DROPOUT': True,
 'filteringsize': (3, 3),
 'imsize': 32,
 'numchans': 3,
 'numclasses': 2,
 'numfilters': 32,
 'numlayers': (4, 2, 1),
 'poolingsize': (2, 2),
 'w_init': None}
(None, 32, 32, 3)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 32)        9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 26, 32)        9248      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
max_pooling2

In [27]:
name = 'MIX'
num_classes=2
num_timewins=5
DROPOUT=True
BIDIRECT=False
cnnseq = iEEGSeq(name=name,
                num_classes=num_classes,
                num_timewins=num_timewins,
                DROPOUT=DROPOUT,
                BIDIRECT=BIDIRECT)
print(cnn.model.output_shape)
cnnseq.buildmodel(cnn.model)
cnnseq.buildoutput()
cnnseq.model.summary()

(None, 1, 1, 128)


../../dnn/model/nets/ieegseq.py:175: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  self.model.add(Merge(convnets, mode='concat'))


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_78 (Conv2D)              (None, 30, 30, 32)   896         input_15[0][0]                   
__________________________________________________________________________________________________
conv2d_79 (Conv2D)              (None, 28, 28, 32)   9248        conv2d_78[0][0]                  
__________________________________________________________________________________________________
conv2d_80 (Conv2D)              (None, 26, 26, 32)   9248        conv2d_79[0][0]                  
__________________________________________________________________________________________________
conv2d_81 

# Show Code For Training Setup

In [3]:
dnnmodel = cnn.model
batch_size = 32
NUM_EPOCHS = 100
AUGMENT = True
cnn_trainer = traincnn.TrainCNN(dnnmodel, batch_size, NUM_EPOCHS, AUGMENT)

In [4]:
cnn_trainer.summaryinfo()

{'augment': True, 'batch_size': 32, 'epochs': 100}


In [11]:
# configure and initialize generator
tempdatadir = './test'
pattraindir = './test'
# list of patients to train on
listofpats_train = [
                'id001',
                'id002', 
                'id008', 
                'id010', 
                'id011', 
                'id012', 
                'id013'
            ]

cnn_trainer.configure(tempdatadir)
cnn_trainer.loadgenerator()
cnn_trainer.loaddirofdata(pattraindir, listofpats_train, LOAD=True)
# cnn_trainer.loaddata(datafile, imsize, n_colors)

In [15]:
testlist = ['hi', 'bye', 'dog']

test = ['hiseq.py', 'dabg', 'byeseq', 'cat', 'catdog']

for item in test:
    if any(x in item for x in testlist):
        print(item)

hiseq.py
byeseq
catdog
